In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, re
from tensorflow import keras
import keras
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn import metrics, preprocessing
from keras.utils.np_utils import to_categorical
from keras.layers import BatchNormalization
from sklearn.utils import shuffle

In [2]:
train_df = pd.read_csv('mixed.csv', sep='\t')
test_df = pd.read_csv("mixed_test.csv", sep='\t')

In [4]:
# choosing space groups with more than 60 samples
fm3m_sample = shuffle(train_df[train_df['space_group'] == 'Fm3m']).iloc[:60]
fm3m_sample = fm3m_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'Space_code']]

r3m_sample = shuffle(train_df[train_df['space_group'] == 'R3m']).iloc[:60]
r3m_sample = r3m_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'Space_code']]

ia3d_sample = shuffle(train_df[train_df['space_group'] == 'Ia3d']).iloc[:60]
ia3d_sample = ia3d_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'Space_code']]

p6_3mmc_sample = shuffle(train_df[train_df['space_group'] == 'P6_3mmc']).iloc[:60]
p6_3mmc_sample = p6_3mmc_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'Space_code']]

In [7]:
peaks = []
codes = []

In [8]:
# getting the sum of two different materials
for i in range(len(fm3m_sample)):
    for j in range(len(r3m_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + r3m_sample.iloc[j][:6].values)
        codes.append(0)
        
for i in range(len(fm3m_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(1)
        
for i in range(len(fm3m_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(2)
        
for i in range(len(fm3m_sample)):
    for j in range(len(fm3m_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + fm3m_sample.iloc[j][:6].values)
        codes.append(3)
        
for i in range(len(r3m_sample)):
    for j in range(len(r3m_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + r3m_sample.iloc[j][:6].values)
        codes.append(4)
        
for i in range(len(r3m_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(5)
        
for i in range(len(r3m_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(6)
        
for i in range(len(ia3d_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(ia3d_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(7)
        
for i in range(len(ia3d_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(ia3d_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(8)
        
for i in range(len(p6_3mmc_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(p6_3mmc_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(9)

In [18]:
data = pd.DataFrame(peaks, columns=['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6'])
data['codes'] = codes

In [19]:
data.head()

,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,codes
0,6.070347,4.381850,3.394297,7.284728,4.075179,7.728272,0
1,7.016713,4.771371,3.774244,2.720819,3.366377,3.006971,0
2,7.047778,4.787809,3.781665,2.727456,2.824725,3.582274,0
3,4.198193,3.701000,2.224401,1.963045,1.914068,2.136740,0
4,4.333584,3.755000,1.602174,2.307098,2.264020,2.653060,0


In [20]:
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,codes
0,5.556681,4.091404,4.501916,4.209752,2.451787,2.150988,1
1,5.212766,3.115224,5.286706,4.546477,2.419685,3.152779,7
2,5.260573,3.143794,5.881500,3.262469,2.093593,4.084726,7
3,4.337921,3.756750,1.603347,2.007628,2.470347,2.144694,3
4,5.541429,4.829944,4.322524,4.729935,2.327353,3.283109,8


In [21]:
data.to_csv("combined.csv", sep='\t', index=False)

In [33]:
fm3m_sample = shuffle(test_df[test_df['space_code'] == 4]).iloc[:60]
fm3m_sample = fm3m_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'space_code']]

r3m_sample = shuffle(test_df[test_df['space_code'] == 7]).iloc[:60]
r3m_sample = r3m_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'space_code']]

ia3d_sample = shuffle(test_df[test_df['space_code'] == 6]).iloc[:60]
ia3d_sample = ia3d_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'space_code']]

p6_3mmc_sample = shuffle(test_df[test_df['space_code'] == 0]).iloc[:60]
p6_3mmc_sample = p6_3mmc_sample[['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'space_code']]

In [34]:
peaks = []
codes = []

In [35]:
for i in range(len(fm3m_sample)):
    for j in range(len(r3m_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + r3m_sample.iloc[j][:6].values)
        codes.append(0)
        
for i in range(len(fm3m_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(1)
        
for i in range(len(fm3m_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(2)
        
for i in range(len(fm3m_sample)):
    for j in range(len(fm3m_sample)):
        peaks.append(fm3m_sample.iloc[i][:6].values + fm3m_sample.iloc[j][:6].values)
        codes.append(3)
        
for i in range(len(r3m_sample)):
    for j in range(len(r3m_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + r3m_sample.iloc[j][:6].values)
        codes.append(4)
        
for i in range(len(r3m_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(5)
        
for i in range(len(r3m_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(r3m_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(6)
        
for i in range(len(ia3d_sample)):
    for j in range(len(ia3d_sample)):
        peaks.append(ia3d_sample.iloc[i][:6].values + ia3d_sample.iloc[j][:6].values)
        codes.append(7)
        
for i in range(len(ia3d_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(ia3d_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(8)
        
for i in range(len(p6_3mmc_sample)):
    for j in range(len(p6_3mmc_sample)):
        peaks.append(p6_3mmc_sample.iloc[i][:6].values + p6_3mmc_sample.iloc[j][:6].values)
        codes.append(9)

In [36]:
data = pd.DataFrame(peaks, columns=['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6'])
data['codes'] = codes

In [37]:
data.head()

,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,codes
0,10.7775,4.5451,8.4836,7.7082,5.6556,7.3987,0
1,10.7694,4.5421,8.4770,7.7036,5.6522,7.3962,0
2,11.2834,9.0859,8.7288,7.9168,5.1537,5.2187,0
3,11.2753,9.0829,8.7222,7.9122,5.1503,5.2162,0
4,11.2809,9.0810,8.7273,7.9142,5.1524,5.2166,0


In [38]:
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,peak_1,peak_2,peak_3,peak_4,peak_5,peak_6,codes
0,7.1102,8.8588,4.8153,6.3630,4.2566,9.4745,2
1,5.2602,3.1153,3.3076,7.9706,4.5926,7.8737,1
2,7.3122,5.2287,6.5021,6.9928,5.2527,8.8917,9
3,6.0249,4.4643,4.5329,4.8186,8.1769,3.9549,1
4,5.7235,4.0331,5.6083,7.0912,3.6638,4.4118,2


In [39]:
data.to_csv("combined_test.csv", sep='\t', index=False)